In [ ]:
!pip install -q langchain-community
!pip install -q langchain_core
!pip install -q gigachain-community
!pip install -q pypdf
!pip install -q chromadb
!pip install -q tiktoken
!pip install -q langchain_experimental
!pip install -q rank_bm25

In [ ]:
import pandas as pd
import re
import numpy as np
from langchain.document_loaders import PyPDFLoader
import langchain_core
from langchain_core.documents.base import Document


document_1 = PyPDFLoader("Положение-Банка-России-от-30-января-2023-г-N-814-П-О-порядке-расчета-размера-опе.pdf").load()
document_2 = PyPDFLoader("Положение-Банка-России-от-7-декабря-2020-г-N-744-П-О-порядке-расчета-размера-опе.pdf").load()
document_3 = PyPDFLoader("Положение-Банка-России-от-8-апреля-2020-г-N-716-П-О-требованиях-к-системе-управл.pdf").load()

##### Для начала будем обрабатывать документ 3

In [ ]:
# чистим от колонтитулов

all_document = ''.join([document_3[i].page_content for i in range(len(document_3))])
print('lenght before cleaning:', len(all_document))
spam = 'Положение Банка России от 8 апреля 2020 г. N 716-П "О требованиях к системе управления операционным риском… \n10.11.2024  Система ГАРАНТ '
all_document = ''.join([chunk for chunk in all_document.split(spam)])
print('lenght after cleaning:', len(all_document))


In [ ]:
# делим на главы и приложения
chapters = all_document.split('Глава ')[1:]
applications = chapters[-1].split('Приложение ')[1:]
chapters[-1] = chapters[-1].split('Приложение ')[0]

whole_doc_dirty = all_document.replace('\n', '')

# делим на пункты
chapters = [re.split('(?:\d\.)+ ', chapter)[1:] for chapter in chapters][:-1]

# chapters = [[chunk for chunk in chapter if len(chunk) > 100] for chapter in chapters]
applications = [re.split('(?:\d\.)+ ', application)[1:] for application in applications][:-1]
# applications = [[chunk for chunk in chapter if len(chunk) > 100] for chapter in applications]


# # объединям все части
whole_doc = [Document(punkt.replace('\n', '')) for chapter in chapters + applications for punkt in chapter]
whole_doc = [doc for doc in whole_doc if len(doc.page_content) > 100]
print('число пунктов в документе:', len(whole_doc))

In [ ]:
def find_number(chunk):
    prefix = whole_doc_dirty.split(chunk)[0]
    return re.findall('(?:\d+\.)+ ', prefix)[-1][:-1]

In [ ]:
for doc in whole_doc:
    doc.metadata = {'header_2':find_number(doc.page_content)}

whole_doc[0].metadata['header_1'] = 1
header = 1
for i, doc in enumerate(whole_doc[1:]):
    if doc.metadata['header_2'][0] != whole_doc[i].metadata['header_2'][0]:
        header += 1
    doc.metadata['header_1'] = header


big_chapter_doc = []
current_header = 1
current_page_content = ''
for doc in whole_doc:
    if doc.metadata['header_1'] == current_header:
        current_page_content += doc.page_content
    else:
        new_doc = Document(current_page_content)
        new_doc.metadata = {'header':current_header}
        big_chapter_doc.append(new_doc)
        current_page_content = doc.page_content
        current_header += 1

new_doc = Document(current_page_content)
new_doc.metadata = {'header':current_header}
big_chapter_doc.append(new_doc)



##### GigaChat API

##### Разбиение на чанки

In [ ]:
import nltk
nltk.download('punkt_tab')

In [ ]:
from langchain.text_splitter import (RecursiveCharacterTextSplitter,
                                    SentenceTransformersTokenTextSplitter,
                                    TokenTextSplitter,
                                    NLTKTextSplitter,
                                    SpacyTextSplitter
                                    )

TOKENS_PER_CHUNK_SIZE = 256
CHUNK_OVERLAP = 0

token_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=CHUNK_OVERLAP, tokens_per_chunk=TOKENS_PER_CHUNK_SIZE)

splitted_docs = token_splitter.split_documents(whole_doc)


In [ ]:
from openai import OpenAI
from google.colab import userdata

LLAMA_TOKEN = userdata.get('LLAMA')


def llama_responce(prompt):
    client = OpenAI(
        base_url="https://openrouter.ai/api/v1",
        api_key=LLAMA_TOKEN,
    )
    completion = client.chat.completions.create(
        model="meta-llama/llama-3.1-70b-instruct:free",
        messages=[
            {
            "role": "user",
            "content": prompt
            }
        ]
    )
    return completion.choices[0].message.content


##### Contextual Chunk Headers

In [ ]:
# тут нужно выбрать более стабильный API и более хорошую модель для суммаризации
summaries = []

# for doc in whole_doc:
#     prompt = "Отвечай как юрист в банковской сфере. Ответ должен содержать не больше 5 предложений. Приведи суммаризацию данного текста.\nТекст: "\
#     + doc.page_content
#     new_doc = Document(llama_responce(prompt))
#     new_doc.metadata = doc.metadata
#     summaries.append(new_doc)

##### Document Augmentation

In [ ]:
# тут нужно выбрать более стабильный API

enriched_docs = []

# for doc in whole_doc:
#     prompt = "Отвечай как юрист в банковской сфере. Приведи пример двух вопросов, которые можно задать по заданному тексту.\nТекст: "\
#     + doc.page_content
#     new_doc = Document(llama_responce(prompt))
#     new_doc.metadata = doc.metadata
#     enriched_docs.append(new_doc)

##### Векторизация

In [ ]:
from langchain.vectorstores import Chroma
import chromadb
import langchain

chroma_client = chromadb.Client()

In [ ]:
from sentence_transformers import SentenceTransformer

# model = SentenceTransformer("deepvk/USER-bge-m3")
# model = SentenceTransformer("intfloat/multilingual-e5-large").to('cuda')
model = SentenceTransformer("intfloat/multilingual-e5-large-instruct").to('cuda')
# model = SentenceTransformer("BAAI/bge-m3").to('cuda')



In [ ]:
class Embedder_wrapper:
    def __init__(self, model):
        self.model = model

    def embed_documents(self, texts):
        return [self.model.encode(text) for text in texts]

    def embed_query(self, query):
        return self.model.encode(query)

In [ ]:
embedder = Embedder_wrapper(model)

In [ ]:
vectordb = Chroma.from_documents(
    documents=splitted_docs,
    embedding=embedder,
    persist_directory='docs/chroma2/'
)

vectordb.persist()

In [ ]:
print(vectordb._collection.count())

##### Загружаем датасет с вопросами

In [ ]:
query_answer = pd.read_excel('queries2.xlsx')
query_answer = query_answer.drop(columns = ['Unnamed: 0'])

queries = list(query_answer['Вопрос'])
answers = list(query_answer['Ответ'])
punkts = [re.findall(r'(?:\d+\.)*\d+', answer) for query, answer in zip(queries, answers)]
queries_df = pd.DataFrame({'query': queries, 'answer' : answers, 'punkts': punkts})
queries_df = queries_df[queries_df['punkts'].apply(lambda x: len(x) == 2)]


##### Считаем метрики для поиска чанка

In [ ]:
def print_metrics_dummy(queries, query_column_name='query', max_ = 10, min_ = 3):
    for k in range(min_, max_+1):
        right = 0
        all = len(queries) * 2
        for row in queries.iterrows():
            right_answers = row[1].loc['punkts']
            query = row[1].loc[query_column_name]
            res = []
            ss = vectordb.max_marginal_relevance_search(query, k)

            for doc in ss:
                res.append(doc.metadata['header'])
            for answer in right_answers:
                if answer + '.' in res:
                    right += 1

        print(f'Recall@{k} =', right / all)

## RAG

In [ ]:
from langchain.schema import HumanMessage, SystemMessage
from langchain.chat_models.gigachat import GigaChat

from google.colab import userdata
API_TOKEN = userdata.get('GIGACHAT')

giga_chat = GigaChat(credentials=API_TOKEN, verify_ssl_certs=False)


#### Fusion Retriever

In [ ]:
import numpy as np

from rank_bm25 import BM25Okapi
from nltk import WordPunctTokenizer

from nltk.corpus import stopwords
nltk.download('stopwords')
russian_stopwords = stopwords.words("russian")

def clean_text(text):
    text = WordPunctTokenizer().tokenize(text)
    text = [token.lower() for token in text if token.isalpha() and token not in russian_stopwords]
    return text


In [ ]:
def fusion_retrieval_block(db, query, alpha=0.9, top_k=10):

    db_retrieved_scores = db.similarity_search_with_relevance_scores(query, len(db))

    clean_texts = [clean_text(doc[0].page_content) for doc in db_retrieved_scores]
    BMdb = BM25Okapi(clean_texts)

    bm25_scores = BMdb.get_scores(clean_text(query))

    vector_scores = np.array([score for _, score in db_retrieved_scores])
    vector_scores = 1 - (vector_scores - np.min(vector_scores)) / (np.max(vector_scores) - np.min(vector_scores))

    bm25_scores = (bm25_scores - np.min(bm25_scores)) / (np.max(bm25_scores) - np.min(bm25_scores))

    combined_scores = alpha * vector_scores + (1 - alpha) * bm25_scores

    sorted_indices = np.argsort(combined_scores)[::-1]

    return [db_retrieved_scores[i] for i in sorted_indices[:top_k]]



#### Rerankers

##### LLM reranker

In [ ]:
def llm_ranker(query, answers, model=giga_chat):

    MODEL_INSTRUCTION = 'Ты юрист в банковской сфере. Отвечай на вопросы на основе Положения Банка России \
"О требованиях к системе управления операционным риском в кредитной организации \
и банковской группе."'

    template = """Ответ должен содержать ровно одно число. Оцени по шкале от 1 до 10, насколько хорошо данный ответ отвечает на заданный вопрос.\
Вопрос: {query}\
Ответ: {answer}"""
    prompt = PromptTemplate(template=template, input_variables=['query', 'answer'])
    scores = []
    for answer in answers:
        PROMPT = prompt.format(query=query, answer=answer.page_content)

        messages = [
            SystemMessage(
                content=MODEL_INSTRUCTION
            ),
            HumanMessage(content=PROMPT)
        ]
        scores.append(float(model(messages)))

        return scores


##### Cross encoder msmacro

In [ ]:
from sentence_transformers import CrossEncoder

reranker_model = CrossEncoder('DiTy/cross-encoder-russian-msmarco', max_length=512, device='cuda')
def cross_encoder_ranker(query, answers):
    answers = [answer.page_content for answer in answers]
    rank_result = reranker_model.rank(query, answers)
    vals = [res['score'] for res in rank_result]
    return vals


#### Rerancer BGE

In [ ]:
# from sentence_transformers import SentenceTransformer
# model = SentenceTransformer('BAAI/bge-large-zh-v1.5')
# embeddings_1 = model.encode(sentences_1, normalize_embeddings=True)




#### RSE

Идея: складываем скоры в каком-то окне вокруг чанка

In [ ]:
# todo

### RAG

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

In [ ]:
def chat(model, prompt):
    MODEL_INSTRUCTION = 'Ты юрист в банковской сфере. Отвечай на вопросы на основе Положения Банка России \
"О требованиях к системе управления операционным риском в кредитной организации \
и банковской группе."'

    messages = [
        SystemMessage(
            content=MODEL_INSTRUCTION
        ),
        HumanMessage(content=prompt)
    ]

    res = model(messages)
    return res.content


In [ ]:
class Retriever:
    def __init__(self, db, reranker=None, strategy='mmr', fusion_alpha=1, k = 10, has_answer_th = 0.):
        self.database = db
        self.reranker = reranker
        self.strategy = strategy
        self.fusion_alpha = fusion_alpha
        self.k = k
        self.has_answer_th = has_answer_th

    def rerank_docs(self, query, docs):
        if self.reranker is None:
            return np.zeros(len(docs))
        return self.reranker(query, docs)

    def retrieve(self, query):
        query = self.get_advance_query(query)
        if self.strategy == 'mmr':
            docs = self.database.max_marginal_relevance_search(query, self.k)
            scores = self.rerank_docs(query, docs)

        else:
            docs, fusion_scores = fusion_retrieval_block(self.database, query, self.fusion_alpha, self.k)
            reranker_scores = self.rerank_docs(docs)
            scores = reranker_scores + fusion_scores

        score_doc = list(zip(scores, docs))
        score_doc.sort(key=lambda x: x[0])
        if score_doc[0][0] < self.has_answer_th:
            return None
        return [elem[1] for elem in score_doc]

    def get_advance_query(self, query): # оболочка для продвинутого класса
        return query


In [ ]:
class EnrichAsAnswerRetriever(Retriever):
    def __init__(self, db, reranker=None, chat_model=None):
        super().__init__(db, reranker)
        self.chat_model = chat_model

    def get_advance_query(self, query):
        PROMPT = 'Приведи пример ответа, который можно найти в Положении Банка России \
"О требованиях к системе управления операционным риском в кредитной организации \
и банковской группе."\n\
Вопрос: Расскажи про учет изменений в иностранном законодательстве при управлении оп риском в зарубежных дочерних кредитных организациях?\n\
Ответ: Кредитная организация должна учитывать требования национального законодательства иностранного государства при управлении операционным риском в дочерних организациях, включая порог регистрации событий. При этом показатели операционного риска приводятся в соответствие с требованиями национального законодательства, если оно противоречит требованиям Положения.\n\
Вопрос: Как кредитная организация должна учитывать потери от реализации событий операционного риска при расчете капитала, и какие требования предъявляются к ведению базы событий в этом контексте?\n\
Ответ: Кредитная организация должна ежемесячно определять величину валовых потерь от реализации событий операционного риска и использовать эти данные при выборе подхода к расчету объема капитала на покрытие таких потерь, выбирая между регуляторным и продвинутым подходами.\n\
Вопрос: '

        new_prompt = PROMPT + query + '\nОтвет: '
        return query + '\n' + chat(self.chat_model, new_prompt)


In [ ]:
class EnrichAsQueryRetriever(Retriever):
    def __init__(self, db, reranker=None, chat_model=None, query_count=3):
        super().__init__(db, reranker)
        self.chat_model = chat_model
        self.query_count = query_count

    def get_advance_query(self, query):
        PROMPT = f'Переформулируй вопрос {self.query_count} способами таким образом, чтобы ответ на них можно было найти в \
Положении Банка России \
"О требованиях к системе управления операционным риском в кредитной организации \
и банковской группе."\n \
Вопрос: '

        new_prompt = PROMPT + query
        return query + '\n' + chat(self.chat_model, new_prompt)


In [ ]:
class EnrichAsCorrectionRetriever(Retriever):
    def __init__(self, db, reranker=None, chat_model=None, query_count=3):
        super().__init__(db, reranker)
        self.chat_model = chat_model
        self.query_count = query_count

    def get_advance_query(self, query):
        PROMPT = f'Ответь одним предложением.\nПереформулируй вопрос так, чтобы он стал более детальным и конкретным. Ответ на вопрос можно найти в \
Положении Банка России \
"О требованиях к системе управления операционным риском в кредитной организации \
и банковской группе."\n \
Вопрос: '

        new_prompt = PROMPT + query
        return query + '\n' + chat(self.chat_model, new_prompt)


In [ ]:
class RAG:
    def __init__(self, retriever, model, verbose = False):
        self.chat_model = model
        self.retriever = retriever


    def get_answer(self, query, template):

        retrieved_documents = self.retriever.retrieve(query)
        if retrieved_documents is None:
            return 'Нет подходящей информации в данном документе'
        docs_page_content = [doc.page_content for doc in retrieved_documents]
        information = "\n\n".join(docs_page_content)
        prompt = PromptTemplate(template=template[0], input_variables=['information', 'query'])

        answer = chat(self.chat_model, prompt.format(information=information, query=query))
        return answer, retrieved_documents


In [ ]:
retriever = EnrichAsCorrectionRetriever(vectordb, reranker=cross_encoder_ranker, chat_model=giga_chat)
rag = RAG(retriever, giga_chat)

tp = """Используй данный контест чтобы ответить на вопрос в конце. Для ответа используй не более двух предложений.\
```{information}```
Вопрос: {query}
Ответ:"""

rag.get_answer(queries[0], tp)

In [ ]:
def measure_function(query, answer, model, is_chat=True):
    if is_chat:

        MODEL_INSTRUCTION = 'Ты юрист в банковской сфере. Отвечай на вопросы на основе Положения Банка России \
"О требованиях к системе управления операционным риском в кредитной организации \
и банковской группе."'

        template = """Ответ должен содержать ровно одно число. Оцени по шкале от 1 до 100, насколько хорошо данный ответ отвечает на заданный вопрос.\
Вопрос: {query}\
Ответ: {answer}"""
        prompt = PromptTemplate(template=template, input_variables=['query', 'answer'])

        PROMPT = prompt.format(query=query, answer=answer)

        messages = [
            SystemMessage(
                content=MODEL_INSTRUCTION
            ),
            HumanMessage(content=PROMPT)
        ]

        res = model(messages)
        return res.content

